In [4]:
# Optional helpers: make JSON-like names safe if accidentally typed unquoted in a cell
try:
    null
except NameError:
    null = None
try:
    true
except NameError:
    true = True
try:
    false
except NameError:
    false = False

In [5]:
from pathlib import Path
import uuid
import re

# No file type restrictions - will rename all files
ALLOWED_EXTENSIONS = None


def prompt_for_directory() -> Path:
    folder_input = input("Enter the path to the folder containing files to rename: ").strip().strip('"').strip("'")
    folder_path = Path(folder_input).expanduser().resolve()
    if not folder_path.exists():
        raise FileNotFoundError(f"Folder does not exist: {folder_path}")
    if not folder_path.is_dir():
        raise NotADirectoryError(f"Not a directory: {folder_path}")
    return folder_path


def prompt_for_semester(default: str = "FALL-2024") -> str:
    raw = input(f"Enter the semester prefix (default '{default}'): ").strip()
    if raw.lower() in {"null", "none"}:
        raw = ""
    value = raw or default
    # Sanitize for Windows-incompatible characters
    value = re.sub(r"[\\\\/:*?\"<>|]", "_", value)
    value = value.strip(" ") or default
    return value


def prompt_for_start_index(default: int = 1) -> int:
    raw = input(f"Enter start index (default {default}): ").strip()
    if raw.lower() in {"null", "none", ""}:
        return default
    try:
        val = int(raw)
        return max(1, val)  # Ensure index starts from 1
    except ValueError:
        return default


def find_all_files(folder: Path):
    all_files = [p for p in folder.iterdir() if p.is_file()]
    # Sort deterministically by filename
    all_files.sort(key=lambda p: p.name.lower())
    return all_files


def rename_files_fall_style(folder: Path, semester: str, start_index: int = 1):
    all_files = find_all_files(folder)
    if not all_files:
        print("No files found to rename.")
        return

    print(f"Found {len(all_files)} file(s). Renaming to {semester}_1..{semester}_{len(all_files)} while preserving extensions...")

    temp_files = []
    # Stage 1: rename everything to unique temporary names to avoid collisions
    for original_path in all_files:
        temp_name = f"{original_path.stem}__TMP_RENAME__{uuid.uuid4().hex}{original_path.suffix}"
        temp_path = original_path.with_name(temp_name)
        original_path.rename(temp_path)
        temp_files.append(temp_path)

    # Stage 2: rename temps to final names
    renamed_paths = []
    try:
        for idx, temp_path in enumerate(temp_files, start=start_index):
            final_name = f"{semester}_{idx}{temp_path.suffix}"
            final_path = folder / final_name
            if final_path.exists():
                raise FileExistsError(f"Final target already exists: {final_path}")
            temp_path.rename(final_path)
            renamed_paths.append(temp_path)
    except Exception as exc:
        print(f"Error during renaming: {exc}. Attempting best-effort rollback...")
        # Best-effort rollback: try to remove the TMP marker from any remaining temp files
        for path in folder.iterdir():
            if path.is_file() and "__TMP_RENAME__" in path.name:
                cleaned_name = path.name.replace("__TMP_RENAME__", "")
                try:
                    path.rename(folder / cleaned_name)
                except Exception:
                    pass
        raise

    print(f"Renamed {len(renamed_paths)} file(s) in: {folder}")
    for preview in renamed_paths[:10]:
        print(f"- {preview.name}")
    if len(renamed_paths) > 10:
        print("...")


if __name__ == "__main__":
    target = prompt_for_directory()
    semester = prompt_for_semester("FALL-2024")
    start = prompt_for_start_index(1)
    rename_files_fall_style(target, semester, start)


Found 49 file(s). Renaming to FALL-2024_1..FALL-2024_49 while preserving extensions...
Renamed 49 file(s) in: D:\Mentor Certificate-(Summer-24_Fall-24)\Fall-24\Mentor-FAll-2024_Updated
- For Academic (1)__TMP_RENAME__2b1ab1d3e74a4dc99df318989023113e.pdf
- For Academic (10)__TMP_RENAME__9844f52199c743d29225d90ac921a5a7.pdf
- For Academic (11)__TMP_RENAME__598dbbe7dcd94e93915d6c52b32535d6.pdf
- For Academic (12)__TMP_RENAME__503ef55f50cd4e0c8be33ede35d32e0d.pdf
- For Academic (13)__TMP_RENAME__c267730f48b84c909e484d081d9cb9bb.pdf
- For Academic (14)__TMP_RENAME__2db1fe32c3c9456199ac07e3fb9e9ad3.pdf
- For Academic (15)__TMP_RENAME__b173ccc6326c435483cddaa8c6dfd2a6.pdf
- For Academic (16)__TMP_RENAME__ea3b3b87593349ae9b40d87f7dcfc10a.pdf
- For Academic (17)__TMP_RENAME__c8d14b4344c146f2a879cb33cf7592dd.pdf
- For Academic (18)__TMP_RENAME__bd4efdfe0e09465ebf08df4eef178745.pdf
...
